In [2]:
import numpy as np
import MDAnalysis as mda
import matplotlib.pyplot as plt
import MDAnalysis.analysis.distances
import networkx as nx
import scipy.stats as stats
from itertools import product as product

In [6]:
ls ../


AnnRevPhysChem_54_141_2003.pdf  from_asb/
EPSRC_survey round4.docx        impl_solv_test/
Multiwfn_3.7.pdf                interface_f8bt/
Untitled.mov                    melt_cooling/
Zoom/                           new_pysoftk/
abstract.docx                   p1d/
amps/                           papers/
analysis-sessions/              pd/
bpc22/                          pysoftk/
cnns/                           pysoftk.zip
exchange/                       reviews/
f8bt/                           rmz/
f8bt_self_assembly/             rmz_thesis_and_declaration 2/
for_chris/                      tests/
for_hrach/                      translocation.yml
for_mesfin/                     tx114_revised.tar
for_natasha/                    ~$view_acs_omega_21.docx
for_raquel/


In [8]:
u=mda.Universe('../for_chris/make_micelle_whole/step5_0.tpr',
               '../for_chris/make_micelle_whole/step5_0.part0002.gro')

In [11]:
np.unique(u.atoms.resnames)

array(['ALA', 'ARG', 'ASN', 'ASP', 'GLN', 'GLU', 'GLY', 'HIS', 'ILE',
       'ION', 'LEU', 'LYS', 'MET', 'PHE', 'PRO', 'SER', 'THR', 'TRP',
       'TYR', 'VAL', 'W'], dtype=object)

In [12]:
###quick clustering check

core = u.select_atoms('protein')
cutoff_cluster = 10


cluster_atoms = core.select_atoms('name '+str(core.atoms.names[0]))

dist_array=MDAnalysis.analysis.distances.contact_matrix(cluster_atoms.atoms.positions,cutoff=cutoff_cluster,box=u.dimensions)

G = nx.from_numpy_matrix(dist_array)
clusters=[h for h in nx.connected_components(G)]

for c in clusters:
    if len(c)==max([len(c_) for c_ in clusters]):
        slab_resids = [i+1 for i in  c]

In [29]:
core = core.select_atoms('resid '+str(' '.join([str(i) for i in slab_resids])))

2880

### "New" method that relies on making fake bonds (only good until certain size)

In [30]:


##make sure the core selection doesn't break up and save positions as slab_calc_positions

# bonds_original=core.bonds

new_bond_atoms = core.select_atoms('name '+str(core.atoms.names[0]))
#just one link
# new_bonds=[((int(new_bond_atoms[i].index)),(int(new_bond_atoms[i+1].index))) for i in range(len(new_bond_atoms)-1)]
#all links
new_bonds=[]
for i in range(len(new_bond_atoms)-1):
    for j in range(i,len(new_bond_atoms)-1):
        new_bonds.append(((int(new_bond_atoms[i].index)),(int(new_bond_atoms[i+1].index))))

u.add_bonds(new_bonds)
core.atoms.unwrap()
core.atoms.write('../../for_chris/test_micelle_unwrapped.gro')


### "Old" method that uses histograms (should work for anything reasonable)

In [13]:
all_atoms=core.copy()
solvent=u.select_atoms('resname W WF ION')
test_poly=core.positions.copy()
water_o_positions=solvent.positions.copy()
box=u.dimensions
for dimension in range(0,3):

    y,a = stats.binned_statistic(all_atoms.positions[:,dimension],all_atoms.positions[:,dimension],bins=np.arange(-50,box[0]+50,10),statistic='count').statistic,stats.binned_statistic(all_atoms.positions[:,dimension],all_atoms.positions[:,dimension],bins=np.arange(-50,box[0]+50,10),statistic='count').bin_edges

    plt.close()
    move_above=999
    move_water=999
    filled=np.where(y!=0)[0]

    for i in range(len(filled)-1):
        if filled[i]!=filled[i+1]-1:
            move_above = a[filled[i+1]]

    for i in range(len(filled)-1):
        if filled[i]!=filled[i+1]-1:
            move_above = a[filled[i+1]]
            move_lower_bound=a[filled[i]]
            move_water=(move_lower_bound+move_above)/2
    ####

    if move_above!=999:
        for atom in range(len(test_poly)):
            if test_poly[atom][dimension]>move_above:
                test_poly[atom][dimension]=test_poly[atom][dimension]-box[dimension]
                
    if move_above!=999:
        for atom in range(len(water_o_positions)):
            if water_o_positions[atom][dimension]>move_water:
                water_o_positions[atom][dimension]=water_o_positions[atom][dimension]-box[dimension]


In [14]:
v=u.copy()

In [15]:
v.select_atoms('not resname W WF ION').positions = test_poly
v.select_atoms('resname W WF ION').positions = water_o_positions

In [17]:
v.select_atoms('not resname W WF ION').positions
v.atoms.write('../for_chris/centered_micelle_11_05_22.gro')